In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [14]:
raw_dir="data/raw/Transactions Data"
interim_dir = "data/interim/prob_turism.csv"

In [15]:
# Raw data to get allocations

frames = []

for filename in os.listdir(raw_dir):
    f = os.path.join(raw_dir, filename)
    print(f)
    # checking if it is a file
    if "detalhado" in f and ".xlsx" in f:
        df = pd.read_excel(f)
        frames.append(df)

raw_data = pd.concat(frames)


data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220718.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220412.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_202110_20220222.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_Agregado_2020_01_2021_11.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220613.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220511.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220818.xlsx
data/raw/Transactions Data/SIBS_Analytics_Freguesias_detalhado_20220310.xlsx


In [16]:
hotel_data = raw_data[(raw_data["Setor"]=="Hoteis e Resorts") | (raw_data["Setor"]=="Alojamento") | (raw_data["Setor"]=="Alojamento Local")].copy()

In [17]:
month_dict = {
    "Jan": 1,
    "Fev": 2,
    "Mar": 3,
    "Abr":4,
    "Mai": 5,
    "Jun": 6,
    "Jul": 7,
    "Ago": 8,
    "Set": 9,
    "Out": 10,
    "Nov": 11,
    "Dez": 12,
}

hotel_data["Mês"] = [month_dict[i[:3]] for i in hotel_data["Ano/Mês"]]
hotel_data["Ano"] = [i[-2:] for i in hotel_data["Ano/Mês"]]
hotel_data = hotel_data.sort_values(by=["Ano", "Mês"])

In [18]:
hotel_data

,Ano/Mês,Tipo de dia,País do cartão,Setor,Freguesia,Nº de operações,Valor das operações,Mês,Ano
28,Nov 21,Dia da Semana,Argentina,Alojamento Local,"União das freguesias de Cedofeita, Santo Ildef...",15,1169.58,11,21
29,Nov 21,Dia da Semana,Argentina,Hoteis e Resorts,"União das freguesias de Cedofeita, Santo Ildef...",20,1213.93,11,21
40,Nov 21,Dia da Semana,Australia,Hoteis e Resorts,"União das freguesias de Cedofeita, Santo Ildef...",20,2209.74,11,21
66,Nov 21,Dia da Semana,Austria,Alojamento,"União das freguesias de Cedofeita, Santo Ildef...",23,1549.79,11,21
67,Nov 21,Dia da Semana,Austria,Alojamento Local,ARU/ORU do Centro Histórico,13,1581.65,11,21
...,...,...,...,...,...,...,...,...,...
19069,Jul 22,Fim de Semana,Estados Unidos Da America,Hoteis e Resorts,ARU/ORU do Centro Histórico,906,153456.06,7,22
19070,Jul 22,Fim de Semana,Estados Unidos Da America,Hoteis e Resorts,Bonfim,100,10977.69,7,22
19071,Jul 22,Fim de Semana,Estados Unidos Da America,Hoteis e Resorts,"União das freguesias de Cedofeita, Santo Ildef...",2034,335563.32,7,22
19072,Jul 22,Fim de Semana,Estados Unidos Da America,Hoteis e Resorts,União das freguesias de Lordelo do Ouro e Mass...,325,32800.71,7,22


In [19]:
# Interim data with prob metric
prob_df = pd.read_csv(interim_dir)